<i>Copyright (c) Microsoft Corporation. All rights reserved.</i>

<i>Licensed under the MIT License.</i>

# Vowpal Wabbit Deep Dive

<center>
<img src="https://github.com/VowpalWabbit/vowpal_wabbit/blob/master/logo_assets/vowpal-wabbits-github-logo.png?raw=true" height="30%" width="30%" alt="Vowpal Wabbit">
</center>

[Vowpal Wabbit](https://github.com/VowpalWabbit/vowpal_wabbit) は、レコメンデーション用途に関連するいくつかのアルゴリズムを実装する高速オンライン機械学習ライブラリです。

Vowpal Wabbit(VW)の主な利点は、トレーニングは通常、確率的勾配降下または類似手法を使用してオンラインで行われ、非常に大きなデータセットにうまくスケーリングできることです。さらに、非常に高速に実行するように最適化されており、非常に大規模なデータセットの分散トレーニング シナリオをサポートできます。

VW は、大きすぎてメモリには収まらないが、単一のノードのディスクであれば格納できるようなデータサイズの問題に最適です。分散トレーニングはノードの追加のセットアップと構成で可能です。VWが適切に扱う問題の種類は、主に機械学習の教師あり分類の領域(線形回帰、ロジスティック回帰、マルチクラス分類、サポートベクターマシン、シンプルニューラルネット)に分類されます。また、行列因子化アプローチと潜在ディリクレット割り当て、およびその他のいくつかのアルゴリズムもサポートしています(詳細については[wiki](https://github.com/VowpalWabbit/vowpal_wabbit/wiki)を参照してください)。

一般的な良い展開例は、ユーザーの広告を掲載するオークションがミリ秒単位で決定されるリアルタイム入札シナリオです。ユーザーとアイテムに関するフィーチャ情報を抽出してモデルに渡し、クリック (またはその他の反応) の可能性を短い時間で予測する必要があります。また、ユーザーとコンテキストのフィーチャーが常に変化している場合 (例えば、ユーザーの利用ブラウザや現地時間)、可能なすべての入力の組み合わせを事前にスコア付けすることは不可能な場合があります。VWは、さまざまなアルゴリズムをオフラインで探索し、大量の履歴データで高精度なモデルをトレーニングし、リアルタイムで迅速な予測を生成できるように、モデルを本番環境に展開するプラットフォームとして価値を提供します。もちろん、これはVWを展開できる唯一の方法ではなく、モデルが絶えず更新されている完全にオンラインな環境で使用したり、アクティブラーニングアプローチを使用したり、事前スコアリングモードで完全にオフラインで作業することも可能です。

<h3>Recommendations 用の Vowpal Wabbit</h3>
このノートブックでは、VW ライブラリを使用して [MovieLens](https://grouplens.org/datasets/movielens/) データセットに関するレコメンデーションを生成する方法を示します。

このノートブックで VW がどのように使用されるかについて、以下の内容に注目してください。

Azure Data Science Virtual Machine ([DSVM](https://azure.microsoft.com/en-us/services/virtual-machines/data-science-virtual-machines/)) を使用すると VW はプリインストールされており、コマンド ラインから直接使用できます。DSVM を使用していない場合は、vw を自分でインストールする必要があります。

また、Python 環境内で VW を使用できるようにする Python バインディングや、SciKit-Learn Estimator API に準拠したラッパーもあります。ただし、Python バインディングは Boost と依存関係を持つ追加の Python パッケージとしてインストールする必要があるため、VW の実行を簡潔にするために、モデルのコマンド ライン実行から動作を模倣するサブプロセス呼び出しを介して行われます。

VWは特定の[入力フォーマット](https://github.com/VowpalWabbit/vowpal_wabbit/wiki/Input-format)を期待しており、このノートブックの to_vw() は、標準的な MovieLens データセットを対応するデータ形式に変換する便利な機能です。その後、データファイルはディスクに書き込まれ、トレーニングのためにVWに渡されます。

以下の例は、異なるアプローチのパフォーマンス上の利点を示さない VW の機能的能力を示すものです。[コマンドライン オプション](https://github.com/VowpalWabbit/vowpal_wabbit/wiki/Command-Line-Arguments)を使用して調整できるVWモデルのパフォーマンスに大きな影響を与えるハイパーパラメータ(学習率や正規化用語など)がいくつかあります。アプローチを適切に比較するには、関連するデータセットについて学習し、調整すると便利でしょう。

# 0. グローバル セットアップ

In [1]:
import sys
sys.path.append('../..')

import os
from subprocess import run
from tempfile import TemporaryDirectory
from time import process_time

import pandas as pd
import papermill as pm

from reco_utils.common.notebook_utils import is_jupyter
from reco_utils.dataset.movielens import load_pandas_df
from reco_utils.dataset.python_splitters import python_random_split
from reco_utils.evaluation.python_evaluation import (rmse, mae, exp_var, rsquared, get_top_k_items,
                                                     map_at_k, ndcg_at_k, precision_at_k, recall_at_k)

print("System version: {}".format(sys.version))
print("Pandas version: {}".format(pd.__version__))

System version: 3.6.8 |Anaconda, Inc.| (default, Dec 30 2018, 01:22:34) 
[GCC 7.3.0]
Pandas version: 0.24.1


In [2]:
def to_vw(df, output, logistic=False):
    """Convert Pandas DataFrame to vw input format
    Args:
        df (pd.DataFrame): input DataFrame
        output (str): path to output file
        logistic (bool): flag to convert label to logistic value
    """
    with open(output, 'w') as f:
        tmp = df.reset_index()

        # vw の書式設定を簡略化するに評価タイプを整数にリセットする必要がある
        tmp['rating'] = tmp['rating'].astype('int64')
        
        # 評価をバイナリ値に変換する
        if logistic:
            tmp['rating'] = tmp['rating'].apply(lambda x: 1 if x >= 3 else -1)
        
        # 各行を VW 入力形式に変換する (https://github.com/VowpalWabbit/vowpal_wabbit/wiki/Input-format) [ラベル]
        # [タグ] |[ユーザー名空間][ユーザー ID 機能] |[項目の名前空間][ムービー ID 機能] ラベルは真の評価であり、
        # タグは予測を真実のユーザーにリンクするために使用される例の一意の ID であり、項目の名前空間は、コマンド ライン
        # オプションを通じて相互作用機能をサポートする機能を分離します。
        for _, row in tmp.iterrows():
            f.write('{rating:d} {index:d}|user {userID:d} |item {itemID:d}\n'.format_map(row))

In [3]:
def run_vw(train_params, test_params, test_data, prediction_path, logistic=False):
    """Convenience function to train, test, and show metrics of interest
    Args:
        train_params (str): vw training parameters
        test_params (str): vw testing parameters
        test_data (pd.dataFrame): test data
        prediction_path (str): path to vw prediction output
        logistic (bool): flag to convert label to logistic value
    Returns:
        (dict): metrics and timing information
    """

    # モデルのトレーニング
    train_start = process_time()
    run(train_params.split(' '), check=True)
    train_stop = process_time()
    
    # モデルのテスト
    test_start = process_time()
    run(test_params.split(' '), check=True)
    test_stop = process_time()
    
    # 予測の読み取り
    pred_df = pd.read_csv(prediction_path, delim_whitespace=True, names=['prediction'], index_col=1).join(test_data)
    pred_df.drop("rating", axis=1, inplace=True)

    test_df = test_data.copy()
    if logistic:
        # メトリックが正しくキャプチャできるように完全なバイナリ ラベルを作成
        test_df['rating'] = test['rating'].apply(lambda x: 1 if x >= 3 else -1)
    else:
        # 結果が正しい範囲の整数であることを確認する
        pred_df['prediction'] = pred_df['prediction'].apply(lambda x: int(max(1, min(5, round(x)))))

    # メトリクスの計算
    result = dict()
    result['RMSE'] = rmse(test_df, pred_df)
    result['MAE'] = mae(test_df, pred_df)
    result['R2'] = rsquared(test_df, pred_df)
    result['Explained Variance'] = exp_var(test_df, pred_df)
    result['Train Time (ms)'] = (train_stop - train_start) * 1000
    result['Test Time (ms)'] = (test_stop - test_start) * 1000
    
    return result

In [4]:
# データ ファイルを管理するための一時ディレクトリの作成
tmpdir = TemporaryDirectory()

model_path = os.path.join(tmpdir.name, 'vw.model')
saved_model_path = os.path.join(tmpdir.name, 'vw_saved.model')
train_path = os.path.join(tmpdir.name, 'train.dat')
test_path = os.path.join(tmpdir.name, 'test.dat')
train_logistic_path = os.path.join(tmpdir.name, 'train_logistic.dat')
test_logistic_path = os.path.join(tmpdir.name, 'test_logistic.dat')
prediction_path = os.path.join(tmpdir.name, 'prediction.dat')
all_test_path = os.path.join(tmpdir.name, 'new_test.dat')
all_prediction_path = os.path.join(tmpdir.name, 'new_prediction.dat')

# 1. データの読み取りと変形

In [5]:
# MovieLens データサイズの選択: 100k, 1m, 10m, or 20m
MOVIELENS_DATA_SIZE = '100k'
TOP_K = 10

In [6]:
# MovieLens データの読み込み
df = load_pandas_df(MOVIELENS_DATA_SIZE)

# トレーニングとテストセット用にデータを分割し、デフォルト値は各ユーザーの評価の75%をトレーニング用として、25%をテスト用として受け取る
train, test = python_random_split(df, 0.75)

# トレーニング、テスト データを vw フォーマットで保存する
to_vw(df=train, output=train_path)
to_vw(df=test, output=test_path)

# ロジスティクス回帰用のデータを保存する (ラベルの調整が必要)
to_vw(df=train, output=train_logistic_path, logistic=True)
to_vw(df=test, output=test_logistic_path, logistic=True)

4.93MB [00:00, 6.18MB/s]                            


# 2. 回帰ベースのレコメンデーション
機械学習の問題を解決するためのさまざまなアプローチを検討する場合、パフォーマンス、時間、およびリソース (メモリまたは CPU) の使用状況全体において、より複雑なソリューションがどのように実行されるかを理解するためのベースライン アプローチを生成すると役立ちます。

回帰ベースのアプローチは、多くの ML 問題に対して考慮すべき最も簡単で最速のベースラインの一つです。

## 2.1 線形回帰
データは 1 から 5 の数値での評価を提供しており、これらの値を線形回帰モデルに適合させるのは簡単なアプローチです。このモデルは、目的変数としての評価の例と、独立した機能としての対応するユーザー ID とムービー ID に関するトレーニングを受けます。

各ユーザー項目の評価を例として渡すことで、モデルは各ユーザーの平均評価とアイテムごとの平均評価に基づいて重みを学習し始めます。

ただし、これは整数でなくなった予測評価を生成できるため、必要に応じて 1 から 5 の整数スケールに戻すには、予測時にいくつかの追加の調整を行う必要があります。ここでは、これを評価関数で行います。

In [7]:
"""
使用されるコマンド ライン パラメータの簡単な説明
  その他のオプションパラメータは、こちらで確認できます: https://github.com/VowpalWabbit/vowpal_wabbit/wiki/Command-Line-Arguments
  VW はデフォルトで線形回帰を使用するので、コマンド ライン オプションはありません。
  -f <model_path>: トレーニング後の最終的なモデル ファイルを保持する場所を示します。
  -d <data_path>: トレーニングまたはテストに使用するデータ ファイルを示します。
  --quiet: これは、Quiet モードで vw を実行します (デバッグの場合は、Quiet モードを使用しないことに役立ちます)
  -i <model_path>: トレーニング中に作成した以前のモデル ファイルを読み込む場所を示します。
  -t: これは推論のみを実行します (モデルに対して学習の更新を行いません)
  -p <prediction_path>: 予測出力を格納する場所を示します。
"""
train_params = 'vw -f {model} -d {data} --quiet'.format(model=model_path, data=train_path)
# 後でトップ K 分析で使用するために結果を保存する
test_params = 'vw -i {model} -d {data} -t -p {pred} --quiet'.format(model=model_path, data=test_path, pred=prediction_path)

result = run_vw(train_params=train_params, 
                test_params=test_params, 
                test_data=test, 
                prediction_path=prediction_path)

comparison = pd.DataFrame(result, index=['Linear Regression'])
comparison

,RMSE,MAE,R2,Explained Variance,Train Time (ms),Test Time (ms)
Linear Regression,0.988433,0.70988,0.227276,0.227286,62.5,15.625


## 2.2 相互作用フィーチャーと線形回帰

これ以前は、ユーザーのフィーチャーとアイテムの機能を個別に扱いましたが、機能間の相互作用を考慮すると、ユーザーのよりきめ細かい環境設定を学習するメカニズムを提供できます。

相互作用機能を生成するには、二次コマンド ライン引数を使用し、各文字の最初の文字に基づいてユーザーと項目の名前空間を組み合わせて結合するオプション '-q ui' を組み合わせて名前空間を指定します。

現在使用される userID および itemID は、フィーチャ ID が直接使用される整数であり、例えばユーザ ID 123 評価ムービー 456 の場合、トレーニング例はフィーチャ 123 および 456 の値に 1 を入れる。ただし、インタラクションが指定されている場合 (またはフィーチャが文字列の場合)、結果として得られるインタラクションフィーチャは使用可能なフィーチャ空間にハッシュされます。フィーチャ ハッシュは、非常に疎な高次元フィーチャ空間を取り込み、より低い次元空間に減らす方法です。これにより、フィーチャとモデルの重み付けの高速計算を維持しながら、メモリを削減できます。

機能ハッシュの注意点は、ハッシュ衝突が発生し、別々のフィーチャが同じ場所にマップされる可能性があることです。この場合、高いカーディナリティのフィーチャ間の相互作用をサポートするためにスペースのサイズを大きくすると有益です。使用可能なフィーチャスペースは --bit_precision (-b) 引数で指定されます。モデル内のすべてのフィーチャーの使用可能な合計スペースは 2<sup>N</sup> です。

詳細については、[フィーチャー ハッシュと抽出](https://github.com/VowpalWabbit/vowpal_wabbit/wiki/Feature-Hashing-and-Extraction)を参照してください。

In [8]:
"""
使用されるコマンド ライン パラメータの簡単な説明
  -b <N>: メモリ サイズを 2<sup>N</sup> エントリに設定します。
  -q <ab>: 'a' と 'b' で始まる名前空間のフィーチャ間の二次フィーチャの相互作用を作成する
"""
train_params = 'vw -b 26 -q ui -f {model} -d {data} --quiet'.format(model=saved_model_path, data=train_path)
test_params = 'vw -i {model} -d {data} -t -p {pred} --quiet'.format(model=saved_model_path, data=test_path, pred=prediction_path)

result = run_vw(train_params=train_params,
                test_params=test_params,
                test_data=test,
                prediction_path=prediction_path)
saved_result = result

comparison = comparison.append(pd.DataFrame(result, index=['Linear Regression w/ Interaction']))
comparison

,RMSE,MAE,R2,Explained Variance,Train Time (ms),Test Time (ms)
Linear Regression,0.988433,0.70988,0.227276,0.227286,62.500,15.625
Linear Regression w/ Interaction,0.985921,0.71292,0.231199,0.231338,15.625,31.250


## 2.3 多項式ロジスティック回帰

線形回帰の代わりに、各評価値を個別のクラスとして扱う多項ロジスティック回帰または多クラス分類を活用します。

これにより、整数以外の結果は回避されますが、異なる評価レベルのカウントが歪んでいるとパフォーマンスが低下する可能性がある各クラスのトレーニング データも減少します。

基本的なマルチクラスロジスティック回帰は、N がクラスの数であり、使用する損失関数のロジスティック オプションを証明する '--oaa N' オプションで指定された 1対全 アプローチを使用して実行できます。

In [9]:
"""
使用されるコマンド ライン パラメータの簡単な説明
  --損失関数ロジスティクス: ロジスティック回帰のモデル損失関数を設定します。
  --oaa <N>: 1対全 アプローチを使用して N の別々のモデルを列車 (すべてのモデルは単一のモデル ファイルにキャプチャされます)
             これにより、ラベルが 1 から始まる連続した整数であることが想定されます。
  --リンク ロジスティクス:予測出力をロジットから確率に変換します。
予測される出力は、最も可能性の高いモデル(ラベル)です。
"""
train_params = 'vw --loss_function logistic --oaa 5 -f {model} -d {data} --quiet'.format(model=model_path, data=train_path)
test_params = 'vw --link logistic -i {model} -d {data} -t -p {pred} --quiet'.format(model=model_path, data=test_path, pred=prediction_path)

result = run_vw(train_params=train_params,
                test_params=test_params,
                test_data=test,
                prediction_path=prediction_path)

comparison = comparison.append(pd.DataFrame(result, index=['Multinomial Regression']))
comparison

,RMSE,MAE,R2,Explained Variance,Train Time (ms),Test Time (ms)
Linear Regression,0.988433,0.70988,0.227276,0.227286,62.500,15.625
Linear Regression w/ Interaction,0.985921,0.71292,0.231199,0.231338,15.625,31.250
Multinomial Regression,1.112780,0.75564,0.020626,0.050111,62.500,31.250


## 2.4 ロジスティック回帰

さらに、ユーザーがアイテムを好きか嫌いかに興味を持つ場合、(1,3] は悪い評価(否定的な結果)で、(3,5] は良い評価(肯定的な結果)などと、入力データをバイナリの結果として表すように調整することができます。

このフレーミングを使用すると、単純なロジスティック回帰モデルを適用できます。ロジスティック回帰を実行するには、loss_function パラメータが'ロジスティック' に変更され、ターゲットラベルが [0, 1] に切り替わります。また、予測中に '--link logistic' を設定して、ロジット出力を確率値に戻してください。

In [10]:
train_params = 'vw --loss_function logistic -f {model} -d {data} --quiet'.format(model=model_path, data=train_logistic_path)
test_params = 'vw --link logistic -i {model} -d {data} -t -p {pred} --quiet'.format(model=model_path, data=test_logistic_path, pred=prediction_path)

result = run_vw(train_params=train_params,
                test_params=test_params,
                test_data=test,
                prediction_path=prediction_path,
                logistic=True)

comparison = comparison.append(pd.DataFrame(result, index=['Logistic Regression']))
comparison

,RMSE,MAE,R2,Explained Variance,Train Time (ms),Test Time (ms)
Linear Regression,0.988433,0.709880,0.227276,0.227286,62.500,15.625
Linear Regression w/ Interaction,0.985921,0.712920,0.231199,0.231338,15.625,31.250
Multinomial Regression,1.112780,0.755640,0.020626,0.050111,62.500,31.250
Logistic Regression,0.717475,0.409551,0.096362,0.142500,46.875,31.250


# 3. 行列因子化に基づくレコメンデーション

上記のアプローチはすべて回帰モデルをトレーニングしますが、VW は 2 つの異なるアプローチで行列因子化もサポートしています。

行列因子化は、回帰モデルをトレーニングする際に特定のユーザー、アイテム、および相互作用に対して直接的な重みを学習するのではなく、ユーザーがアイテムを評価する方法を決定する潜在因子を学習しようとします。これがどのように機能するかの例としては、ジャンル別にユーザー設定と項目分類を表すことができる場合があります。ジャンルのセットが小さい場合は、各項目が各ジャンルクラスにどれだけ属しているかを関連付け、各ジャンルのユーザーの好みに合った重みを設定できます。重みの両方のセットは、内部製品がユーザー項目の評価になるベクトルとして表すことができる。行列因子化アプローチでは、ユーザーの潜在フィーチャと項目の低ランク行列を学習し、これらの行列を組み合わせて元のユーザー項目行列を近似できるようにします。

## 3.1. 単数形値分解を基にした行列因子化

最初のアプローチでは、特異値分解(SVD)に基づいて行列因子化を実行し、ユーザ項目評価行列の低ランク近似を学習します。これは '--rank' コマンドライン引数を使用して呼び出されます。

詳細については、[行列因子化の例](https://github.com/VowpalWabbit/vowpal_wabbit/wiki/Matrix-factorization-example)を参照してください。

In [11]:
"""
使用されるコマンド ライン パラメータの簡単な説明
  --ランク <N>: 減少行列内の潜在因子の数を設定します。
"""
train_params = 'vw --rank 5 -q ui -f {model} -d {data} --quiet'.format(model=model_path, data=train_path)
test_params = 'vw -i {model} -d {data} -t -p {pred} --quiet'.format(model=model_path, data=test_path, pred=prediction_path)

result = run_vw(train_params=train_params,
                test_params=test_params,
                test_data=test,
                prediction_path=prediction_path)

comparison = comparison.append(pd.DataFrame(result, index=['Matrix Factorization (Rank)']))
comparison

,RMSE,MAE,R2,Explained Variance,Train Time (ms),Test Time (ms)
Linear Regression,0.988433,0.709880,0.227276,0.227286,62.500,15.625
Linear Regression w/ Interaction,0.985921,0.712920,0.231199,0.231338,15.625,31.250
Multinomial Regression,1.112780,0.755640,0.020626,0.050111,62.500,31.250
Logistic Regression,0.717475,0.409551,0.096362,0.142500,46.875,31.250
Matrix Factorization (Rank),1.010723,0.745800,0.192033,0.221080,31.250,15.625


## 3.2. ファクタリングマシンベースの行列因子化

 [Rendel のファクタリング マシン](https://cseweb.ucsd.edu/classes/fa17/cse291-b/reading/Rendle2010FM.pdf)に基づく別のアプローチは、'--lrq' (低ランク二次) を使用して呼び出されます。このデモの LRQ の詳細について詳しくは、こちらをご覧ください。

これにより、ユーザー項目評価行列の近似値を生成するために乗算される 2 つの下位ランク行列が学習されます。この方法で行列を圧縮すると、非常に疎な相互作用機能を持つ回帰モデルを使用する場合の制限の一部を回避する汎用的な要因を学習できます。これにより、コンバージェンスが向上し、ディスク上のモデルが小さくなる可能性があります。

パフォーマンスを向上させる追加用語は --lrqdropout で、トレーニング中に列を削除します。ただし、これは最適なランク サイズを大きくする傾向があります。L2 正則化などの他のパラメーターは、オーバーフィットを回避するのに役立ちます。

In [12]:
"""
使用されるコマンド ライン パラメータの簡単な説明
  --lrq <abN>: 'a' と 'b' で始まる名前空間との間の二次相互作用に対するランク N の近似値を学習します。
  --lrqdroupout: 一般化を改善するためにトレーニング中にドロップアウトを実行します。
"""
train_params = 'vw --lrq ui7 -f {model} -d {data} --quiet'.format(model=model_path, data=train_path)
test_params = 'vw -i {model} -d {data} -t -p {pred} --quiet'.format(model=model_path, data=test_path, pred=prediction_path)

result = run_vw(train_params=train_params,
                test_params=test_params,
                test_data=test,
                prediction_path=prediction_path)

comparison = comparison.append(pd.DataFrame(result, index=['Matrix Factorization (LRQ)']))
comparison

,RMSE,MAE,R2,Explained Variance,Train Time (ms),Test Time (ms)
Linear Regression,0.988433,0.709880,0.227276,0.227286,62.500,15.625
Linear Regression w/ Interaction,0.985921,0.712920,0.231199,0.231338,15.625,31.250
Multinomial Regression,1.112780,0.755640,0.020626,0.050111,62.500,31.250
Logistic Regression,0.717475,0.409551,0.096362,0.142500,46.875,31.250
Matrix Factorization (Rank),1.010723,0.745800,0.192033,0.221080,31.250,15.625
Matrix Factorization (LRQ),1.013627,0.726640,0.187383,0.187388,31.250,31.250


# 4. 結論

上の表は、推奨予測に使用できる VW ライブラリのアプローチの一部を示しています。相対的なパフォーマンスは、異なるデータセットに適用され、適切に調整されると変わりますが、すべてのアプローチがトレーニングできる速度 (75,000 例) とテスト (25,000 例) に注目すると便利です。

# 5. スコアリング

上記のいずれかのアプローチでモデルをトレーニングした後、モデルを使用して、オフライン バッチ モードまたはリアルタイム スコアリング モードで潜在的なユーザー ペアをスコア付けできます。次の例は、reco_utils ディレクトリのユーティリティを活用して、オフラインスコア付け出力から Top-K のレコメンデーションを生成する方法を示しています。

In [13]:
# 最初に、各ユーザーのすべての項目 (トレーニング中に見られるものを除く) のテスト セットを構築します

users = df[['userID']].drop_duplicates()
users['key'] = 1

items = df[['itemID']].drop_duplicates()
items['key'] = 1

all_pairs = pd.merge(users, items, on='key').drop(columns=['key'])

# トレーニングデータと組み合わせて、トレーニングでメモされたエントリのみを保持する
merged = pd.merge(train[['userID', 'itemID', 'rating']], all_pairs, on=["userID", "itemID"], how="outer")
all_user_items = merged[merged['rating'].isnull()].fillna(0).astype('int64')

# vw形式で保存(これはしばらく時間がかかる場合があります)
to_vw(df=all_user_items, output=all_test_path)

In [14]:
# 新しいデータセットで保存されたモデル (相互作用を伴う線形回帰) を実行する
test_start = process_time()
test_params = 'vw -i {model} -d {data} -t -p {pred} --quiet'.format(model=saved_model_path, data=all_test_path, pred=prediction_path)
run(test_params.split(' '), check=True)
test_stop = process_time()
test_time = test_stop - test_start

# 予測を読み込み、以前に保存した結果からトップkを取得
pred_data = pd.read_csv(prediction_path, delim_whitespace=True, names=['prediction'], index_col=1).join(all_user_items)
top_k = get_top_k_items(pred_data, col_rating='prediction', k=TOP_K)[['prediction', 'userID', 'itemID']]
top_k.head()

,prediction,userID,itemID
0,4.565871,1,318
1,4.533308,1,64
2,4.530738,1,408
3,4.525889,1,603
4,4.501398,1,483


In [15]:
# ランキング メトリックを取得する
args = [test, top_k]
kwargs = dict(col_user='userID', col_item='itemID', col_rating='rating', col_prediction='prediction',
              relevancy_method='top_k', k=TOP_K)

rank_metrics = {'MAP': map_at_k(*args, **kwargs), 
                'NDCG': ndcg_at_k(*args, **kwargs),
                'Precision': precision_at_k(*args, **kwargs),
                'Recall': recall_at_k(*args, **kwargs)}

In [16]:
# 最終結果
all_results = ['{k}: {v}'.format(k=k, v=v) for k, v in saved_result.items()]
all_results += ['{k}: {v}'.format(k=k, v=v) for k, v in rank_metrics.items()]
print('\n'.join(all_results))

RMSE: 0.9859208893212478
MAE: 0.71292
R2: 0.23119931215379363
Explained Variance: 0.2313379575958101
Train Time (ms): 15.625
Test Time (ms): 31.25
MAP: 0.012535184652143394
NDCG: 0.0965940631559385
Precision: 0.0977707006369427
Recall: 0.03761253544606115


# 6. クリーンアップ

In [17]:
# テストの結果を記録
if is_jupyter():
    pm.record('rmse', saved_result['RMSE'])
    pm.record('mae', saved_result['MAE'])
    pm.record('rsquared', saved_result['R2'])
    pm.record('exp_var', saved_result['Explained Variance'])
    pm.record("train_time", saved_result['Train Time (ms)'])
    pm.record("test_time", test_time)
    pm.record('map', rank_metrics['MAP'])
    pm.record('ndcg', rank_metrics['NDCG'])
    pm.record('precision', rank_metrics['Precision'])
    pm.record('recall', rank_metrics['Recall'])

In [18]:
tmpdir.cleanup()

## 参考文献

1. John Langford, et. al. Vowpal Wabbit Wiki. URL: https://github.com/VowpalWabbit/vowpal_wabbit/wiki
2. Steffen Rendel. Factorization Machines. 2010 IEEE International Conference on Data Mining.
3. Jake Hoffman. Matrix Factorization Example. URL: https://github.com/VowpalWabbit/vowpal_wabbit/wiki/Matrix-factorization-example
4. Paul Minero. Low Rank Quadratic Example. URL: https://github.com/VowpalWabbit/vowpal_wabbit/tree/master/demo/movielens